In [1]:
# https://github.com/dragen1860/TensorFlow-2.x-Tutorials/blob/master/04-Linear-Regression/main.py

In [2]:
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
import  os

2021-09-17 02:29:14.096595: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
class Regressor(keras.layers.Layer):

    def __init__(self):
        super(Regressor, self).__init__()

        # here must specify shape instead of tensor !
        # name here is meanless !
        # [dim_in, dim_out]
        self.w = self.add_variable('meanless-name', [13, 1])
        # [dim_out]
        self.b = self.add_variable('meanless-name', [1])

        print(self.w.shape, self.b.shape)
        print(type(self.w), tf.is_tensor(self.w), self.w.name)
        print(type(self.b), tf.is_tensor(self.b), self.b.name)


    def call(self, x):

        x = tf.matmul(x, self.w) + self.b

        return x

In [4]:
def main():

    tf.random.set_seed(22)
    np.random.seed(22)
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    assert tf.__version__.startswith('2.')


    (x_train, y_train), (x_val, y_val) = keras.datasets.boston_housing.load_data()
    #
    print(x_train.shape, y_train.shape, type(x_train), type(y_train))
    x_train, x_val = x_train.astype(np.float32), x_val.astype(np.float32)
    # (404, 13) (404,) (102, 13) (102,)
    print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)
    # Here has two mis-leading issues:
    # 1. (x_train, y_train) cant be written as [x_train, y_train]
    # 2.
    db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64)
    db_val = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(102)


    model = Regressor()
    criteon = keras.losses.MeanSquaredError()
    optimizer = keras.optimizers.Adam(learning_rate=1e-2)

    for epoch in range(200):

        for step, (x, y) in enumerate(db_train):

            with tf.GradientTape() as tape:
                # [b, 1]
                logits = model(x)
                # [b]
                #print(logits.shape)
                logits = tf.squeeze(logits, axis=1)
                #print(logits.shape)
                # [b] vs [b]
                loss = criteon(y, logits)

            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        print(epoch, 'loss:', loss.numpy())


        if epoch % 10 == 0:

            for x, y in db_val:
                # [b, 1]
                logits = model(x)
                # [b]
                logits = tf.squeeze(logits, axis=1)
                # [b] vs [b]
                loss = criteon(y, logits)

                print(epoch, 'val loss:', loss.numpy())
if __name__ == '__main__':
    main()

57344/57026 [==============================] - 0s 0us/step
(404, 13) (404,) <class 'numpy.ndarray'> <class 'numpy.ndarray'>
(404, 13) (404,) (102, 13) (102,)


2021-09-17 02:29:20.657501: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-17 02:29:20.660798: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-17 02:29:20.712262: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-17 02:29:20.712919: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-17 02:29:20.712975: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-17 02:29:20.747838: I tensorflow/stream_executor/platform/def

(13, 1) (1,)
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> True meanless-name:0
<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> True meanless-name:0


2021-09-17 02:29:22.659013: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-17 02:29:23.461316: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


0 loss: 39694.82
0 val loss: 36500.332
1 loss: 18709.125
2 loss: 6534.4346
3 loss: 1271.5068
4 loss: 51.908894
5 loss: 236.74033
6 loss: 401.17242
7 loss: 293.07904
8 loss: 127.253456
9 loss: 48.899597
10 loss: 42.268055
10 val loss: 124.677574
11 loss: 49.233555
12 loss: 47.356434
13 loss: 42.244324
14 loss: 39.96755
15 loss: 39.8223
16 loss: 39.752235
17 loss: 39.398033
18 loss: 39.110874
19 loss: 38.935944
20 loss: 38.738983
20 val loss: 120.3187
21 loss: 38.48421
22 loss: 38.20948
23 loss: 37.93725
24 loss: 37.667336
25 loss: 37.395905
26 loss: 37.119164
27 loss: 36.833477
28 loss: 36.537876
29 loss: 36.234398
30 loss: 35.92582
30 val loss: 114.442184
31 loss: 35.61399
32 loss: 35.299515
33 loss: 34.982445
34 loss: 34.662853
35 loss: 34.34115
36 loss: 34.01806
37 loss: 33.694305
38 loss: 33.370476
39 loss: 33.047012
40 loss: 32.72424
40 val loss: 107.79624
41 loss: 32.402504
42 loss: 32.082157
43 loss: 31.76353
44 loss: 31.446949
45 loss: 31.13268
46 loss: 30.820965
47 loss: 30.512